In [19]:
import xarray as xr
import pint
import pint_xarray
import pandas as pd
import numpy as np
from openscm_units import unit_registry as ureg

pint_xarray.accessors.setup_registry(ureg)

# Minimal example

contains only the required metadata, with multiple gases and gwps

In [63]:
# create with dummy data; note that usually we would read data from a file or API, which looks cleaner
time = pd.date_range('2000-01-01', '2020-01-01', freq='AS')
area_iso3 = np.array(['COL', 'ARG', 'MEX', 'BOL'])
minimal = xr.Dataset({
    ent: xr.DataArray(
        data=np.random.rand(len(time), len(area_iso3), 1),
        coords={'time': time, 'area (ISO3)': area_iso3, 'source': ['RANDMIKA2020']},
        dims=['time', 'area (ISO3)', 'source'],
        attrs={'units': f'{ent} Gg / year', 'entity': ent}
    )
    for ent in ('CO2', 'SF6', 'CH4')
},
    attrs={'area': 'area (ISO3)'}).pint.quantify(unit_registry=ureg)

with ureg.context('SARGWP100'):
    minimal['SF6 (SARGWP100)'] = minimal['SF6'].pint.to('CO2 Gg / year')
minimal['SF6 (SARGWP100)'].attrs['gwp_context'] = 'SARGWP100'

In [64]:
minimal

Magnitude,[[[0.7195054232258915] [0.23998808238333513] [0.6252577121381027] [0.7024561378287788]] [[0.5258731127614179] [0.44719662761653145] [0.3831175335499909] [0.04397951886759166]] [[0.6655035476012071] [0.3961728598120191] [0.587803031842282] [0.7764333746895681]] [[0.14970849906696415] [0.1215258924723035] [0.7515500994509333] [0.14991573252430057]] [[0.3810986610344439] [0.3140562232143421] [0.6083930324408086] [0.8152477788234589]] [[0.19579853545007087] [0.8346379021350746] [0.9698019523149076] [0.791438327199052]] [[0.8922326059964778] [0.14303692960767922] [0.21789630066864496] [0.6218483098966221]] [[0.7678029961991016] [0.42129901287041815] [0.8956226220594711] [0.3519177329744547]] [[0.7413044839373585] [0.038157712114860964] [0.4828237574636014] [0.531526524966382]] [[0.1903929413589639] [0.1608245745720539] [0.7819926832900926] [0.769037658504895]] [[0.3498829707724389] [0.8885521720387853] [0.7238936777356046] [0.45948328611396694]] [[0.43050748434527464] [0.7559594457662042] [0.6944307215336069] [0.7234185753818856]] [[0.3816181022950532] [0.9089114908640851] [0.9296617092954195] [0.23360398134526017]] [[0.3064860345868897] [0.8330454424299535] [0.203512035108821] [0.3154788836169031]] [[0.08717005933105448] [0.6767560453205025] [0.09997438243285761] [0.006280573046013815]] [[0.3451675113272663] [0.6671068582471086] [0.21534833138891718] [0.18487184690563752]] [[0.5721893524274753] [0.2799887111898962] [0.3639641810757649] [0.5888994606931399]] [[0.00676055659408803] [0.5843381552263182] [0.4318506237571855] [0.848976526672254]] [[0.7244629932356782] [0.475489707890119] [0.545347540997974] [0.7221999937316542]] [[0.7740575005472414] [0.766091054574704] [0.34737693408862935] [0.426674377908445]] [[0.6731982025621203] [0.021772015038147963] [0.9730161090566064] [0.7191962832617321]]]
Units,CO2 gigagram/year
Magnitude,[[[0.6345570744965248] [0.8091110324958245] [0.25899474519859345] [0.6678342431563745]] [[0.6306299660908875] [0.9157915277706524] [0.009779096088089245] [0.04700050093787389]] [[0.33455807962912876] [0.24385980563544785] [0.8905773402942022] [0.636660353897636]] [[0.3451287483793338] [0.8189199816521312] [0.08149624053084303] [0.886994399675483]] [[0.9290577832725999] [0.020071986023155075] [0.23508206478021598] [0.3006738140839458]] [[0.49512565939760955] [0.2473885533427841] [0.7917744058358721] [0.5674540179231442]] [[0.8861212553966678] [0.05150685193977955] [0.7601053113343031] [0.5000274679805149]] [[0.25328874973025284] [0.7859243032512355] [0.2663080878475337] [0.24521277731833524]] [[0.6312028133581489] [0.9420098125691313] [0.7517214871124928] [0.6965375416311118]] [[0.2911382264245378] [0.1773029885647348] [0.008806720970693016] [0.8800396650849611]] [[0.5215451517053578] [0.9995222384970401] [0.6825070581355671] [0.38882034693277356]] [[0.1621058165457171] [0.5185359796785406] [0.2493089114639898] [0.2245470467656997]] [[0.047882530395113254] [0.2824898891177714] [0.21542764570084305] [0.2380688857827168]] [[0.31116774551285475] [0.8073847146206516] [0.7633507937997204] [0.6112959810132977]] [[0.3308297117006711] [0.973283073785487] [0.6189132029997665] [0.05923564717770968]] [[0.15242073870906014] [0.9737759633023142] [0.32546111268390365] [0.1726286179079184]] [[0.7660975500707922] [0.28270263626036096] [0.0387791951151889] [0.2582515616506412]] [[0.3021245581283102] [0.6862790657333021] [0.8633926747630942] [0.15151665473463716]] [[0.7741668247305881] [0.641467528268835] [0.9162901761617559] [0.7433780641267888]] [[0.44093447126084917] [0.24235208421805532] [0.03164546650020461] [0.12762976255752623]] [[0.7508909056122484] [0.9912012394179925] [0.9019023730243482] [0.7783474028064901]]]
Units,SF6 gigagram/year
Magnitude,[[[0.32003194018929204] [0.13075334859188792] [0.8312072942536504] [0.18987590880368543]] [[0.8288712016424752] [0.36630768188840124] [0.9396590842488679] [0.033718916662112575]] [[0.17225603304749515] [0.13871654743895145] [0.8393755329343544] [0.5928090926517002]]

# Opulent example

contains every metadata and different dimensionalities

In [80]:
# create with dummy data
coords = {
    'time': pd.date_range('2000-01-01', '2020-01-01', freq='AS'),
    'area (ISO3)': np.array(['COL', 'ARG', 'MEX', 'BOL']),
    'category (IPCC 2006)': np.array(['0', '1', '2', '3', '4', '5', '1.A', '1.B']),
    'animal (FAOSTAT)': np.array(['cow', 'swine', 'goat']),
    'scenario (FAOSTAT)': np.array(['highpop', 'lowpop']),
    'provenance': np.array(['projected']),
    'model': np.array(['FANCYFAO']),
    'source': np.array(['RANDMIKA2020']),
}

opulent = xr.Dataset({
    ent: xr.DataArray(
        data=np.random.rand(*(len(x) for x in coords.values())),
        coords=coords,
        dims=list(coords.keys()),
        attrs={'units': f'{ent} Gg / year', 'entity': ent}
    )
    for ent in ('CO2', 'SF6', 'CH4')
},
    attrs={'area': 'area (ISO3)',
           'cat': 'category (IPCC 2006)',
           'sec_cats': ['animal (FAOSTAT)'],
           'scen': 'scenario (FAOSTAT)',
           'reference': {'doi': '10.1012', 'publication': 'FAOSTAT 2020…'},
           'rights': 'Use only with written permission by the secretary general, otherwise blue helmets will hunt you down.',
           'contact': 'lol_no_one_will_answer@example.com',
           'description': 'GHG inventory data for selected countries, by IPCC 2006 category and animal. For convenience, population data is also provided'}
)

pop_coords = {x: coords[x] for x in ('time', 'area (ISO3)', 'provenance', 'model', 'source')}
opulent['population'] = xr.DataArray(
    data=np.random.rand(*(len(x) for x in pop_coords.values())),
    coords=pop_coords,
    dims=list(pop_coords.keys()),
    attrs={'entity': 'population', 'units': ''},
)

opulent = opulent.assign_coords({'category names': xr.DataArray(
    data=np.array(['total', 'industry', 'energy', 'transportation', 'residential', 'land use', 'heavy industry', 'light industry']),
    coords={'category (IPCC 2006)': coords['category (IPCC 2006)']},
    dims=['category (IPCC 2006)'],
)})

opulent = opulent.pint.quantify(unit_registry=ureg)

with ureg.context('SARGWP100'):
    opulent['SF6 (SARGWP100)'] = opulent['SF6'].pint.to('CO2 Gg / year')
opulent['SF6 (SARGWP100)'].attrs['gwp_context'] = 'SARGWP100'


In [81]:
opulent

<xarray.Dataset>
Dimensions:               (animal (FAOSTAT): 3, area (ISO3): 4, category (IPCC 2006): 8, model: 1, provenance: 1, scenario (FAOSTAT): 2, source: 1, time: 21)
Coordinates:
  * time                  (time) datetime64[ns] 2000-01-01 ... 2020-01-01
  * area (ISO3)           (area (ISO3)) <U3 'COL' 'ARG' 'MEX' 'BOL'
  * category (IPCC 2006)  (category (IPCC 2006)) <U3 '0' '1' '2' ... '1.A' '1.B'
  * animal (FAOSTAT)      (animal (FAOSTAT)) <U5 'cow' 'swine' 'goat'
  * scenario (FAOSTAT)    (scenario (FAOSTAT)) <U7 'highpop' 'lowpop'
  * provenance            (provenance) <U9 'projected'
  * model                 (model) <U8 'FANCYFAO'
  * source                (source) <U12 'RANDMIKA2020'
    category names        (category (IPCC 2006)) <U14 'total' ... 'light indu...
Data variables:
    CO2                   (time, area (ISO3), category (IPCC 2006), animal (FAOSTAT), scenario (FAOSTAT), provenance, model, source) float64 <Quantity([[[[[[[[0.21075735]]]        [[[0.33713797]]]]        [[[[0.40176279]]]        [[[0.75195426]]]]        [[[[0.91581204]]]        [[[0.13967172]]]]]        [[[[[0.25799245]]]        [[[0.9892193 ]]]]        [[[[0.27220861]]]        [[[0.11558032]]]]        [[[[0.67710158]]]        [[[0.17260243]]]]]        [[[[[0.09269891]]]        [[[0.63450768]]]]        [[[[0.56764071]]]        [[[0.99664782]]]]        [[[[0.58935002]]]        [[[0.91758567]]]]]        ...        [[[[[0.12784681]]]        [[[0.65112505]]]]        [[[[0.48061547]]]        [[[0.30167574]]]]        [[[[0.78363718]]]        [[[0.94646978]]]]]        [[[[[0.90795156]]]        [[[0.08236139]]]]        [[[[0.64029835]]]        [[[0.67052287]]]]        [[[[0.24458362]]]        [[[0.02054038]]]]]        [[[[[0.15297156]]]        [[[0.9044186 ]]]]        [[[[0.23558301]]]        [[[0.11923069]]]]        [[[[0.77735287]]]        [[[0.86295748]]]]]]        [[[[[[0.97804196]]]        [[[0.47673487]]]]        [[[[0.31825516]]]        [[[0.35353777]]]]        [[[[0.11973683]]]        [[[0.00427261]]]]]        [[[[[0.14235556]]]        [[[0.0594727 ]]]]        [[[[0.20843158]]]        [[[0.1005058 ]]]]        [[[[0.43075675]]]        [[[0.10819753]]]]]        [[[[[0.97228884]]]        [[[0.40407159]]]]        [[[[0.8892181 ]]]        [[[0.10827052]]]]        [[[[0.54115385]]]        [[[0.80816339]]]]]        ...        [[[[[0.0606053 ]]]        [[[0.09067957]]]]        [[[[0.04431081]]]        [[[0.56059423]]]]        [[[[0.14248824]]]        [[[0.75812785]]]]]        [[[[[0.60726087]]]        [[[0.91090168]]]]        [[[[0.3777593 ]]]        [[[0.34876179]]]]        [[[[0.17534042]]]        [[[0.50385785]]]]]        [[[[[0.90761874]]]        [[[0.75574571]]]]        [[[[0.37646279]]]        [[[0.69548296]]]]        [[[[0.70148832]]]        [[[0.2994177 ]]]]]]        [[[[[[0.29287787]]]        [[[0.69079705]]]]        [[[[0.38359541]]]        [[[0.33715974]]]]        [[[[0.5777632 ]]]        [[[0.05472089]]]]]        [[[[[0.12404618]]]        [[[0.41048833]]]]        [[[[0.73342863]]]        [[[0.8036318 ]]]]        [[[[0.43371914]]]        [[[0.20249206]]]]]        [[[[[0.89852269]]]        [[[0.03872542]]]]        [[[[0.215564  ]]]        [[[0.28151992]]]]        [[[[0.06256206]]]        [[[0.95376246]]]]]        ...        [[[[[0.15599456]]]        [[[0.84670744]]]]        [[[[0.8038912 ]]]        [[[0.24234523]]]]        [[[[0.46633856]]]        [[[0.82759343]]]]]        [[[[[0.95738407]]]        [[[0.06991392]]]]        [[[[0.13799833]]]        [[[0.94205791]]]]        [[[[0.45824258]]]        [[[0.90859478]]]]]        [[[[[0.65030237]]]        [[[0.45696365]]]]        [[[[0.39161732]]]        [[[0.18217783]]]]        [[[[0.43955991]]]        [[[0.76757013]]]]]]        [[[[[[0.75273465]]]        [[[0.08787871]]]]        [[[[0.42270325]]]        [[[0.80537441]]]]        [[[[0.52932037]]]        [[[0.73581371]]]]]        [[[[[0.76035224]]]        [[[0.01183046]]]]        [[[[0.78403879]]]        [[[0.08165999]]]]        [[[[0.11843

In [86]:
opulent[['CO2']]

<xarray.Dataset>
Dimensions:               (animal (FAOSTAT): 3, area (ISO3): 4, category (IPCC 2006): 8, model: 1, provenance: 1, scenario (FAOSTAT): 2, source: 1, time: 21)
Coordinates:
  * category (IPCC 2006)  (category (IPCC 2006)) <U3 '0' '1' '2' ... '1.A' '1.B'
  * scenario (FAOSTAT)    (scenario (FAOSTAT)) <U7 'highpop' 'lowpop'
  * source                (source) <U12 'RANDMIKA2020'
  * area (ISO3)           (area (ISO3)) <U3 'COL' 'ARG' 'MEX' 'BOL'
  * time                  (time) datetime64[ns] 2000-01-01 ... 2020-01-01
  * animal (FAOSTAT)      (animal (FAOSTAT)) <U5 'cow' 'swine' 'goat'
  * model                 (model) <U8 'FANCYFAO'
  * provenance            (provenance) <U9 'projected'
    category names        (category (IPCC 2006)) <U14 'total' ... 'light indu...
Data variables:
    CO2                   (time, area (ISO3), category (IPCC 2006), animal (FAOSTAT), scenario (FAOSTAT), provenance, model, source) float64 <Quantity([[[[[[[[0.21075735]]]        [[[0.33713797]]]]        [[[[0.40176279]]]        [[[0.75195426]]]]        [[[[0.91581204]]]        [[[0.13967172]]]]]        [[[[[0.25799245]]]        [[[0.9892193 ]]]]        [[[[0.27220861]]]        [[[0.11558032]]]]        [[[[0.67710158]]]        [[[0.17260243]]]]]        [[[[[0.09269891]]]        [[[0.63450768]]]]        [[[[0.56764071]]]        [[[0.99664782]]]]        [[[[0.58935002]]]        [[[0.91758567]]]]]        ...        [[[[[0.12784681]]]        [[[0.65112505]]]]        [[[[0.48061547]]]        [[[0.30167574]]]]        [[[[0.78363718]]]        [[[0.94646978]]]]]        [[[[[0.90795156]]]        [[[0.08236139]]]]        [[[[0.64029835]]]        [[[0.67052287]]]]        [[[[0.24458362]]]        [[[0.02054038]]]]]        [[[[[0.15297156]]]        [[[0.9044186 ]]]]        [[[[0.23558301]]]        [[[0.11923069]]]]        [[[[0.77735287]]]        [[[0.86295748]]]]]]        [[[[[[0.97804196]]]        [[[0.47673487]]]]        [[[[0.31825516]]]        [[[0.35353777]]]]        [[[[0.11973683]]]        [[[0.00427261]]]]]        [[[[[0.14235556]]]        [[[0.0594727 ]]]]        [[[[0.20843158]]]        [[[0.1005058 ]]]]        [[[[0.43075675]]]        [[[0.10819753]]]]]        [[[[[0.97228884]]]        [[[0.40407159]]]]        [[[[0.8892181 ]]]        [[[0.10827052]]]]        [[[[0.54115385]]]        [[[0.80816339]]]]]        ...        [[[[[0.0606053 ]]]        [[[0.09067957]]]]        [[[[0.04431081]]]        [[[0.56059423]]]]        [[[[0.14248824]]]        [[[0.75812785]]]]]        [[[[[0.60726087]]]        [[[0.91090168]]]]        [[[[0.3777593 ]]]        [[[0.34876179]]]]        [[[[0.17534042]]]        [[[0.50385785]]]]]        [[[[[0.90761874]]]        [[[0.75574571]]]]        [[[[0.37646279]]]        [[[0.69548296]]]]        [[[[0.70148832]]]        [[[0.2994177 ]]]]]]        [[[[[[0.29287787]]]        [[[0.69079705]]]]        [[[[0.38359541]]]        [[[0.33715974]]]]        [[[[0.5777632 ]]]        [[[0.05472089]]]]]        [[[[[0.12404618]]]        [[[0.41048833]]]]        [[[[0.73342863]]]        [[[0.8036318 ]]]]        [[[[0.43371914]]]        [[[0.20249206]]]]]        [[[[[0.89852269]]]        [[[0.03872542]]]]        [[[[0.215564  ]]]        [[[0.28151992]]]]        [[[[0.06256206]]]        [[[0.95376246]]]]]        ...        [[[[[0.15599456]]]        [[[0.84670744]]]]        [[[[0.8038912 ]]]        [[[0.24234523]]]]        [[[[0.46633856]]]        [[[0.82759343]]]]]        [[[[[0.95738407]]]        [[[0.06991392]]]]        [[[[0.13799833]]]        [[[0.94205791]]]]        [[[[0.45824258]]]        [[[0.90859478]]]]]        [[[[[0.65030237]]]        [[[0.45696365]]]]        [[[[0.39161732]]]        [[[0.18217783]]]]        [[[[0.43955991]]]        [[[0.76757013]]]]]]        [[[[[[0.75273465]]]        [[[0.08787871]]]]        [[[[0.42270325]]]        [[[0.80537441]]]]        [[[[0.52932037]]]        [[[0.73581371]]]]]        [[[[[0.76035224]]]        [[[0.01183046]]]]        [[[[0.78403879]]]        [[[0.08165999]]]]        [[[[0.11843

In [83]:
opulent[['CO2', 'SF6', 'CH4', 'SF6 (SARGWP100)']].to_dataframe()

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order)


CO2  \
animal (FAOSTAT) area (ISO3) category (IPCC 2006) model    provenance scenario (FAOSTAT) source       time                   
cow              COL         0                    FANCYFAO projected  highpop            RANDMIKA2020 2000-01-01  0.210757   
                                                                                                      2001-01-01  0.119250   
                                                                                                      2002-01-01  0.488580   
                                                                                                      2003-01-01  0.683356   
                                                                                                      2004-01-01  0.964612   
...                                                                                                                    ...   
goat             BOL         1.B                  FANCYFAO projected  lowpop             RANDMIKA2020 2016-01-01  0.464463   
                                                                                                      2017-01-01  0.370557   
                                                                                                      2018-01-01  0.727943   
                                                                                                      2019-01-01  0.458266   
                                                                                                      2020-01-01  0.058926   

                                                                                                                       SF6  \
animal (FAOSTAT) area (ISO3) category (IPCC 2006) model    provenance scenario (FAOSTAT) source       time                   
cow              COL         0                    FANCYFAO projected  highpop            RANDMIKA2020 2000-01-01  0.980810   
                                                                                                      2001-01-01  0.006936   
                                                                                                      2002-01-01  0.748151   
                                                                                                      2003-01-01  0.813539   
                                                                                                      2004-01-01  0.151976   
...                                                                                                                    ...   
goat             BOL         1.B                  FANCYFAO projected  lowpop             RANDMIKA2020 2016-01-01  0.707902   
                                                                                                      2017-01-01  0.395677   
                                                                                                      2018-01-01  0.791386   
                                                                                                      2019-01-01  0.587888   
                                                                                                      2020-01-01  0.133684   

                                                                                                                       CH4  \
animal (FAOSTAT) area (ISO3) category (IPCC 2006) model    provenance scenario (FAOSTAT) source       time                   
cow              COL         0                    FANCYFAO projected  highpop            RANDMIKA2020 2000-01-01  0.767108   
                                                                                                      2001-01-01  0.022239   
                                                                                                      2002-01-01  0.237716   
                                                                                                      2003-01-01  0.880990   
                                                                                     

In [84]:
# open questions: could standardize on scalar dimensions for len-1 dimensions, which would save some brackets but would also mean they are slightly
# different

In [85]:
# ex. for scalar dimensions:
opulent.loc[{'model': 'FANCYFAO', 'source': 'RANDMIKA2020', 'provenance': 'projected'}]

<xarray.Dataset>
Dimensions:               (animal (FAOSTAT): 3, area (ISO3): 4, category (IPCC 2006): 8, scenario (FAOSTAT): 2, time: 21)
Coordinates:
  * time                  (time) datetime64[ns] 2000-01-01 ... 2020-01-01
  * area (ISO3)           (area (ISO3)) <U3 'COL' 'ARG' 'MEX' 'BOL'
  * category (IPCC 2006)  (category (IPCC 2006)) <U3 '0' '1' '2' ... '1.A' '1.B'
  * animal (FAOSTAT)      (animal (FAOSTAT)) <U5 'cow' 'swine' 'goat'
  * scenario (FAOSTAT)    (scenario (FAOSTAT)) <U7 'highpop' 'lowpop'
    provenance            <U9 'projected'
    model                 <U8 'FANCYFAO'
    source                <U12 'RANDMIKA2020'
    category names        (category (IPCC 2006)) <U14 'total' ... 'light indu...
Data variables:
    CO2                   (time, area (ISO3), category (IPCC 2006), animal (FAOSTAT), scenario (FAOSTAT)) float64 <Quantity([[[[[0.21075735 0.33713797]     [0.40176279 0.75195426]     [0.91581204 0.13967172]]     [[0.25799245 0.9892193 ]     [0.27220861 0.11558032]     [0.67710158 0.17260243]]     [[0.09269891 0.63450768]     [0.56764071 0.99664782]     [0.58935002 0.91758567]]     ...     [[0.12784681 0.65112505]     [0.48061547 0.30167574]     [0.78363718 0.94646978]]     [[0.90795156 0.08236139]     [0.64029835 0.67052287]     [0.24458362 0.02054038]]     [[0.15297156 0.9044186 ]     [0.23558301 0.11923069]     [0.77735287 0.86295748]]]     [[[0.97804196 0.47673487]     [0.31825516 0.35353777]     [0.11973683 0.00427261]]     [[0.14235556 0.0594727 ]     [0.20843158 0.1005058 ]     [0.43075675 0.10819753]]     [[0.97228884 0.40407159]     [0.8892181  0.10827052]     [0.54115385 0.80816339]]     ...     [[0.0606053  0.09067957]     [0.04431081 0.56059423]     [0.14248824 0.75812785]]     [[0.60726087 0.91090168]     [0.3777593  0.34876179]     [0.17534042 0.50385785]]     [[0.90761874 0.75574571]     [0.37646279 0.69548296]     [0.70148832 0.2994177 ]]]     [[[0.29287787 0.69079705]     [0.38359541 0.33715974]     [0.5777632  0.05472089]]     [[0.12404618 0.41048833]     [0.73342863 0.8036318 ]     [0.43371914 0.20249206]]     [[0.89852269 0.03872542]     [0.215564   0.28151992]     [0.06256206 0.95376246]]     ...     [[0.15599456 0.84670744]     [0.8038912  0.24234523]     [0.46633856 0.82759343]]     [[0.95738407 0.06991392]     [0.13799833 0.94205791]     [0.45824258 0.90859478]]     [[0.65030237 0.45696365]     [0.39161732 0.18217783]     [0.43955991 0.76757013]]]     [[[0.75273465 0.08787871]     [0.42270325 0.80537441]     [0.52932037 0.73581371]]     [[0.76035224 0.01183046]     [0.78403879 0.08165999]     [0.11843803 0.42976955]]     [[0.58950857 0.59760348]     [0.31949065 0.42824674]     [0.17395753 0.54819418]]     ...     [[0.95152661 0.82468167]     [0.49961465 0.85333653]     [0.42444375 0.1439277 ]]     [[0.07681432 0.24846362]     [0.04933165 0.80550505]     [0.77045616 0.43132408]]     [[0.7490015  0.59418673]     [0.46137015 0.12054217]     [0.68271821 0.37179394]]]]     [[[[0.11924959 0.39863813]     [0.63425641 0.63871912]     [0.14839594 0.96103182]]     [[0.4175092  0.54585131]     [0.71271744 0.90884734]     [0.14277807 0.29273559]]     [[0.6688076  0.94015836]     [0.05397409 0.33054337]     [0.87710663 0.83335654]]     ...     [[0.3510456  0.6246017 ]     [0.7676316  0.75265554]     [0.15750966 0.43058969]]     [[0.13659177 0.23482246]     [0.53558243 0.43419801]     [0.9481203  0.54782676]]     [[0.4714308  0.28122785]     [0.516418   0.3904546 ]     [0.90645485 0.57029459]]]     [[[0.83887618 0.29796248]     [0.71349681 0.33768813]     [0.72241008 0.58287715]]     [[0.45024638 0.88116398]     [0.97508087 0.22252562]     [0.58996296 0.39626467]]     [[0.30919871 0.52542026]     [0.37767431 0.95810866]     [0.42393325 0.57891347]]     ...     [[0.50924974 0.49702549]     [0.54489859 0.03702348]     [0.54190928 0.31111984]]     [[0.43531713 0.66750312]     [0.09714036 0.05696771]     [0.79550041 0.02348978]]     [[0.68789182 0.12151277]     [0.83214096 0.90452674]     [0.26